#### Motivation
* Create feature layer of businesses by joining san diego biz directory listing and businesses from ca_business (google scraped)

In [1]:
import pandas as pd
import geopandas
import matplotlib.pyplot as plt
import sqlalchemy as sal
import psycopg2
import arcgis
from arcgis.gis import GIS
from arcgis.features import FeatureLayer, FeatureLayerCollection
import sys
sys.path.append('../../')
from gis_resources import san_diego_county_zips
import os
from utils import get_config

In [2]:
from arcgis.features import GeoAccessor

In [4]:
gis = GIS("https://ucsdonline.maps.arcgis.com/home", client_id=get_config("arcgis","clientid"))

Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://ucsdonline.maps.arcgis.com/sharing/rest/oauth2/authorize?response_type=code&client_id=Elm5V3upnnV17Q3r&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=nJR4Dg6QXRvRz3haB6MWgntD3ALFKs&allow_verification=false


Enter code obtained on signing in using SAML:  ········


/Users/amolkale/Documents/DSE/DSE260-Capstone-Project/capstone/venv3.9/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ucsdonline.maps.arcgis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [5]:
nourish_user = get_config("nourish_db","username")
nourish_pswd = get_config("nourish_db","passkey")

In [18]:
conn = psycopg2.connect(
    host="awesome-hw.sdsc.edu",
    database="nourish",
    user=nourish_user,
    password=nourish_pswd)

In [19]:
# create a cursor
cur = conn.cursor()
        

# execute a statement
cur.execute('select * from "SD_City_Business_Directory"')

        # display the PostgreSQL database server version
sd_city_biz_dir_results = cur.fetchall()
       
	# close the communication with the PostgreSQL
cur.close()

In [20]:
sd_city_biz_dir_list = [(row[1],row[2],row[3],row[4],row[5],row[6],row[13]) for row in sd_city_biz_dir_results]

In [23]:
sd_city_biz_dataFrame = pd.DataFrame(sd_city_biz_dir_list,
              columns=("business_name","ownership_type", "address","city", "zip","state","business_type")
              );
print(f"Shape: {sd_city_biz_dataFrame.shape}")
sd_city_biz_dataFrame.head(4)

Shape: (50755, 7)


,business_name,ownership_type,address,city,zip,state,business_type
0,#1 FIFTH AVE,LLC,3845 05TH AVE,SAN DIEGO,92103-3140,CA,DRINKING PLACES (ALCOHOLIC BEVERAGES)
1,% NATURAL GOMEZ,SOLE,4356 POPLAR ST,SAN DIEGO,92105-4657,CA,ALL OTHER HEALTH & PERSONAL CARE STORES
2,& SALLY J FERNANDEZ,SOLE,17471 PLAZA CERADO SUITE 98,SAN DIEGO,92128-2285,CA,OTHER ACCOUNTING SERVICES
3,(W)RIGHT ON COMMUNICATIONS INC,SCORP,402 W BROADWAY SUITE 2700,SAN DIEGO,92101-8580,CA,PUBLIC RELATIONS AGENCIES


In [21]:
sd_city_biz_dir_list[1]

('% NATURAL GOMEZ',
 'SOLE',
 '4356 POPLAR ST',
 'SAN DIEGO',
 '92105-4657',
 'CA',
 'ALL OTHER HEALTH & PERSONAL CARE STORES')

In [14]:
# create a cursor
cur = conn.cursor()
        

# execute a statement
cur.execute('select * from ca_business')

        # display the PostgreSQL database server version
ca_business_result = cur.fetchall()
       
	# close the communication with the PostgreSQL
cur.close()

In [15]:
ca_business_result_list = [(row[1],row[4],row[5],row[7],row[8],row[9]) for row in ca_business_result]
#lat_long_list[:10]

In [16]:
# Create a DataFrame
# There could be some limitation to keeping `categories` as column since arcgis was messing up the data
# , could it be a keyword ? Let's just change the name to 'business_type'
dataFrame = pd.DataFrame(ca_business_result_list,
              columns=("business_name","latitude", "longitude","zip", "business_type","city")
              );

In [17]:
dataFrame.head(2)

,business_name,latitude,longitude,zip,business_type,city
0,Alley 6 Craft Distillery,38.6281104,-122.8759307,95448.0,[Distillery],Healdsburg
1,Amami Sushi Bistro,37.6201856,-122.410203,94066.0,[Sushi restaurant],San Bruno


In [18]:
# since some of the zip are not present in ca_business table
dataFrame = dataFrame[~dataFrame['zip'].isna()]

In [19]:
dataFrame = dataFrame.astype({"zip": int})

In [20]:
# Pick the data for only San Diego County
san_diego_county_zips_lst = san_diego_county_zips()
san_diego_county_zips_lst[:10]

[91910, 91911, 91913, 91914, 91915, 91916, 91917, 91931, 91932, 91934]

In [21]:
dataFrame = dataFrame[dataFrame["zip"].isin(san_diego_county_zips_lst)]
dataFrame.head(2)

,business_name,latitude,longitude,zip,business_type,city
4,Internet Solutions For Less,33.1863539,-117.25029649999999,92081,"[Website designer, Design agency, Internet mar...",Vista
23,Wetzel's Pretzels,32.708565199999995,-117.17027739999999,92101,[Pretzel store],San Diego


In [22]:
# Converting the dataframe with lat/long column to geo dataframe using geopandas
gdf = geopandas.GeoDataFrame(dataFrame, geometry=geopandas.points_from_xy(dataFrame.longitude, dataFrame.latitude))

In [32]:
ca_business_sgdf = GeoAccessor.from_geodataframe(gdf, inplace=False, column_name='SHAPE')

In [26]:
#ca_business_sgdf.info()

In [34]:
#ca_business_sgdf.head(5)
ca_business_sgdf['business_type'].fillna('',inplace=True)
ca_business_sgdf['business_type']=ca_business_sgdf.business_type.apply(lambda x: ';'.join(x))
#ca_business_sgdf

In [35]:
ca_business_sgdf=ca_business_sgdf.drop(['latitude','longitude'], axis=1)

In [36]:
ca_business_sgdf['zip'] = ca_business_sgdf['zip'].astype(str)

In [37]:
ca_business_sgdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46516 entries, 4 to 515948
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   business_name  46516 non-null  object  
 1   zip            46516 non-null  object  
 2   business_type  46516 non-null  object  
 3   city           46381 non-null  object  
 4   SHAPE          46516 non-null  geometry
dtypes: geometry(1), object(4)
memory usage: 2.1+ MB


In [40]:
print(f"Shape of Dataframe: {ca_business_sgdf.shape}")

Shape of Dataframe: (46516, 5)


In [58]:
%%time
# Convert back from a SEDF into a feature layer Collection, and publishing on AGOL
feature_layer_collection_item = ca_business_sgdf.spatial.to_featurelayer(title="San Diego County Businesses 1.0", 
                                                         gis=gis, 
                                                         folder='nourish_gis',
                                                         tags=['SanDiegoCountyBusinesses'],
                                                        )

CPU times: user 37.1 s, sys: 2.79 s, total: 39.9 s
Wall time: 2min 3s


In [59]:
feature_layer_collection_item

<Item title:"San Diego County Businesses 1.0" type:Feature Layer Collection owner:akale_UCSDOnline>